In [2]:
import importlib
import sys
from tqdm import tqdm
import  os
import glob
from datetime import datetime
import matplotlib.pyplot as plt
# Add the directories to the sys.path
dir_path = "C://Users//jony//Programming//Python//Anubis//anubis//" # insert your directory path
sys.path.append(dir_path + "Osiris//processing//python")
sys.path.append(dir_path + "tools")

import Analysis_tools as ATools
import proAnubis_Analysis_Tools
import Reconstruction_tools as RTools
import mplhep as hep
import Timing_tools as TTools
import rawFileReader
import Visual_tools as VTools
hep.style.use([hep.style.ATLAS])

# Specify the directory
data_list = sorted([f for f in os.listdir("data//reset") if os.path.isfile(os.path.join("data//reset", f))], reverse=True) ##all files in data directory sorted from the newest to the oldest

file_path = list(map(lambda p: dir_path+"data//reset//"+p, data_list))[:1] #
print(file_path)

['C://Users//jony//Programming//Python//Anubis//anubis//data//reset//proAnubis_240811_2240.raw']


In [5]:
importlib.reload(rawFileReader) # Reload fReader
importlib.reload(proAnubis_Analysis_Tools)
importlib.reload(ATools)
importlib.reload(VTools)
import matplotlib.pyplot as plt

#['C://Users//jony//Programming//Python//Anubis//anubis//data//reset//proAnubis_240811_2240.raw'] # only 0,1 channels
#file_path = paths = ['C://Users//jony//Programming//Python//Anubis//anubis//data//proAnubis_240731_0217.raw', 'C://Users//jony//Programming//Python//Anubis//anubis//data//reset//proAnubis_240810_0040.raw', 'C://Users//jony//Programming//Python//Anubis//anubis//data//proAnubis_240810_0840.raw']


file = file_path[0]
file_name = file.split("//")[-1][:-4]
print(file_name)
fReader = rawFileReader.fileReader(file)
fReader.skip_events(100_000)
max_chunks = 10

proccesedChunk = 0
bcr_time = 0
trigger_count = 0
bcr_count = 0
time_window = 30 #in seconds
triggers_rate = []
dates = []
channels = (1,2) #(1,0)
tdc5Reads = []
#maybe later against time
time = []
with tqdm(total=max_chunks, desc=f"Processing {file_name}", unit='Chunks') as pbar:
    while proccesedChunk < max_chunks:
        initial_time = None
        trigger_count = 0
        bcr_count = 0        
        time_collected = 0
        while time_collected < time_window:
            if not fReader.readBlock():
                print("Finished Reading File")
                max_chunks = proccesedChunk
                break

            if(fReader.hasEvents()):
                tdc5Reads = fReader.getTDCFiveEvents()
                if not tdc5Reads:
                    continue
                if not initial_time:
                    initial_time = tdc5Reads[0][0]
                    print("Initial Time:", initial_time)
                for word in tdc5Reads[0][1]:
                    tdcChannel = (word>>24)&0x7f
                    tdcHitTime = word&0xfffff
                    """
                    if tdcChannel == channels[1]:
                        bcr_count += 1
                        check = tdcHitTime-bcr_time
                        if check:
                            bcr_time = tdcHitTime
                            print("--------------------")
                            print("BCR Update",bcr_time)
                            print("Check:", check)
                    """
                    if tdcChannel == channels[0]:
                        trigger_count += 1
                    time_collected = (tdc5Reads[0][0]-initial_time).total_seconds()
                        
                
        #print("BCR Count:", br_count)
        #print("Trigger Count:", triggers_count)
        if initial_time:
            time_diff = (tdc5Reads[0][0]-initial_time).total_seconds()
        if tdc5Reads and time_diff:
            dates.append(tdc5Reads[0][0])
            triggers_rate.append(trigger_count/(time_diff))
            proccesedChunk += 1
            pbar.update(1)
            

plt.plot(dates, triggers_rate, linestyle='solid', label=f"{file_name}")
plt.xlabel("Date")
plt.ylim(0, 8000)
plt.legend()
plt.ylabel("Triggers Rate (Hz)")
plt.xticks(dates[0::2], [d.strftime("%H:%M:%S") for d in dates[0::2]])
plt.title(f"Triggers Rate")
plt.show()                    
                 
        
    

proAnubis_240811_2240


Processing proAnubis_240811_2240:   0%|          | 0/10 [00:00<?, ?Chunks/s]

Initial Time: 2024-08-11 21:40:52.235534


Processing proAnubis_240811_2240:  10%|█         | 1/10 [00:27<04:06, 27.40s/Chunks]

Initial Time: 2024-08-11 21:41:22.247916


...

Processing proAnubis_240811_2240:  80%|████████  | 8/10 [03:57<00:59, 29.95s/Chunks]

Initial Time: 2024-08-11 21:44:52.308820


Processing proAnubis_240811_2240:  90%|█████████ | 9/10 [04:25<00:29, 29.28s/Chunks]

Initial Time: 2024-08-11 21:45:22.309079


Processing proAnubis_240811_2240: 100%|██████████| 10/10 [04:58<00:00, 29.83s/Chunks]


In [3]:
plt.plot(dates, triggers_rate, linestyle='solid', label=f"{file_name}")
plt.xlabel("Date")
plt.ylim(0, 8000)
plt.legend()
plt.ylabel("Triggers Rate (Hz)")
plt.xticks(dates[0::2], [d.strftime("%H:%M:%S") for d in dates[0::2]])
plt.title(f"Triggers Rate")
plt.show()

NameError: name 'dates' is not defined

In [57]:
#Data Cleanup
import Atlas_tools as AtTools
importlib.reload(AtTools)
file = 'C://Users//jony//Programming//Python//Anubis//anubis//data//proAnubis_240627_1720.raw'
analyser = AtTools.AtlasAnalyser(file, trigger_channel=4,bcr_channel=2)
analyser.getTDC5Data(10)
#analyser.printEvents(10)


Initial Time: 2024-06-27 16:20:18.092231
BCR(737854*25/32 ns, 2024-06-27 16:20:18.088189, 2024-06-27 16:20:18.088100,1) 
    Triggers: [])
Check: 179584
BCR(917438*25/32 ns, 2024-06-27 16:20:18.088278, 2024-06-27 16:20:18.088100,2) 
    Triggers: [])
Check: 48512
BCR(965950*25/32 ns, 2024-06-27 16:20:18.088367, 2024-06-27 16:20:18.088100,3) 
    Triggers: ['Trigger(982398*25/32 ns, 0)', 'Trigger(995372*25/32 ns, 2024-06-27 16:20:18.088367)'])
Check: -934544
BCR(31406*25/32 ns, 2024-06-27 16:20:18.088456, 2024-06-27 16:20:18.088100,4) 
    Triggers: ['Trigger(131423*25/32 ns, 0)'])
Check: 114129
BCR(145535*25/32 ns, 2024-06-27 16:20:18.088545, 2024-06-27 16:20:18.088100,5) 
    Triggers: [])
Check: 113983
BCR(259518*25/32 ns, 2024-06-27 16:20:18.088634, 2024-06-27 16:20:18.088100,6) 
    Triggers: ['Trigger(270335*25/32 ns, 0)', 'Trigger(274981*25/32 ns, 2024-06-27 16:20:18.088634)'])
Check: 114049
BCR(373567*25/32 ns, 2024-06-27 16:20:18.088723, 2024-06-27 16:20:18.088100,7) 
    Trigg